In [6]:
#Evaluate a model (saved during training in the 'models' directory) 
#through Monte Carlo analysis over the random parameter space for the performance indicators 
#'Successful Landing', 'Collection Ratio', 'Collection Ratio and Landed' as defined in the paper 
#(plus 'Boundary Counter' counting safety controller activations), e.g. for 1000 Monte Carlo iterations:

import argparse
import os

import numpy as np

import import_ipynb
from MainEnvironment import EnvironmentParams, Environment
from Utils import read_config

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tensorflow as tf


def eval_logs(event_path):
    event_acc = EventAccumulator(event_path, size_guidance={'tensors': 100000})
    event_acc.Reload()

    _, _, vals = zip(*event_acc.Tensors('successful_landing'))
    has_landed = [tf.make_ndarray(val) for val in vals]

    _, _, vals = zip(*event_acc.Tensors('cr'))
    cr = [tf.make_ndarray(val) for val in vals]

    _, _, vals = zip(*event_acc.Tensors('cral'))
    cral = [tf.make_ndarray(val) for val in vals]

    _, _, vals = zip(*event_acc.Tensors('boundary_counter'))
    boundary_counter = [tf.make_ndarray(val) for val in vals]

    print("Successful Landing:", sum(has_landed) / len(has_landed))
    print("Collection ratio:", sum(cr) / len(cr))
    print("Collection ratio and landed:", sum(cral) / len(cral))
    print("Boundary counter:", sum(boundary_counter) / len(boundary_counter))


def mc(args, params: EnvironmentParams):
    if args.num_agents is not None:
        num_range = [int(i) for i in args.num_agents]
        params.grid_params.num_agents_range = num_range

    try:
        env = Environment(params)
        env.agent.load_weights(args.weights)

        env.eval(int(args.samples), show=args.show)
    except AttributeError:
        print("Not overriding log dir, eval existing:")

    eval_logs("logs/training/" + args.id + "/test")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    #parser.add_argument('--weights', required=True, help='Path to weights')
    parser.add_argument('--weights', default='models/grid_image_best', help='Path to weights')
    parser.add_argument('--config', default='config/grid_image.json', help='Config file for agent shaping')
    #parser.add_argument('--config', required=True, help='Config file for agent shaping')
    #parser.add_argument('--id', required=False, help='Id for exported files')
    parser.add_argument('--id', default = 'grid_image', help='Id for exported files')
    parser.add_argument('--samples', default = 1000, help='Id for exported files')
    #parser.add_argument('--samples', required=1000, help='Id for exported files')
    parser.add_argument('--seed', default=None, help="Seed for repeatability")
    parser.add_argument('--show', default=False, help="Show individual plots, allows saving")
    parser.add_argument('--num_agents', default=None, help='Overrides num agents range, argument 12 for range [1,2]')

    #args = parser.parse_args()
    args = parser.parse_args(args=[])

    if args.seed:
        np.random.seed(int(args.seed))

    params = read_config(args.config)

    if args.id is not None:
        params.model_stats_params.save_model = "models/" + args.id
        params.model_stats_params.log_file_name = args.id

    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    mc(args, params)

Parse Params file here from  config/grid_image.json  and pass into main
logs/training/grid_image already exists.
Override log file? [Y/n]
1
Deleting old log dir


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
boolean_map_input (InputLayer)  [(None, 174, 174, 4) 0                                            
__________________________________________________________________________________________________
tf.cast_6 (TFOpLambda)          (None, 174, 174, 4)  0           boolean_map_input[0][0]          
__________________________________________________________________________________________________
float_map_input (InputLayer)    [(None, 174, 174, 2) 0                                            
__________________________________________________________________________________________________
tf.concat_1 (TFOpLambda)        (None, 174, 174, 6)  0           tf.cast_6[0][0]                  
                                                                 float_map_input[0][0]     

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [21:32<00:00,  1.29s/it]


Successful Landing: 0.954
Collection ratio: 0.01111491577210628
Collection ratio and landed: 1.9230538392633888e-07
Boundary counter: 0.056
